# Intro to Intervals 

Current notebooks demonstrates main operations that can be performed over intervals

# Imports

In [1]:
from OSOL.Extremum.Numerical_Objects.Interval import *

from OSOL.Extremum.Tools.Encoders import CustomEncoder as encoder

/Users/wol4aravio/.local/share/virtualenvs/Examples-cWHzaMCb/lib/python3.6/site-packages/OSOL/Extremum/Numerical_Objects/Interval.py:23: UserWarning: Can not parse config.ini file. Setting default __MIN_WIDTH = 1e-7
  warnings.warn('Can not parse config.ini file. Setting default __MIN_WIDTH = 1e-7')


In [2]:
import json

# Creation of interval

### Via constructor

In [3]:
i1 = Interval(lower_bound=-1.0, upper_bound=1.0)
i2 = Interval(lower_bound=-3.0, upper_bound=5.0)

print('Interval i1: {}'.format(i1))
print('Interval i2: {}'.format(i2))

Interval i1: [-1.0; 1.0]
Interval i2: [-3.0; 5.0]


### From ordinary number

In [4]:
i3 = Interval.from_value(2.0)

print('Interval i3: {}'.format(i3))

Interval i3: [2.0; 2.0]


### From dictionary

In [5]:
i4 = Interval.from_dict({'lower_bound': 0.0, 'upper_bound': 3.0})

print('Interval i4: {}'.format(i4))

Interval i4: [0.0; 3.0]


### Serialization

OSOL.Extremum supports interval serialization to JSON (via custom encoder)

In [6]:
j = json.dumps(i1, cls=encoder)

print(j)

{"Interval": {"lower_bound": -1.0, "upper_bound": 1.0}}


In [7]:
i5 = Interval.from_json(json.loads(j))

print(i5)

[-1.0; 1.0]


### Valid Interval Creation

For numerical computation it is not required to have intervals with arbitrary small width (especially during the solution of optimal control tasks).

Method `Interval.create_valid_interval` can be used for this purpose. It checks whether desired interval exceeds allowed value of width that is controlled via `Interval.__MIN_WIDTH` constant.

In [14]:
print('Interval.__MIN_WIDTH: {}'.format(Interval._Interval__MIN_WIDTH))

Interval.__MIN_WIDTH: 0.001


Intervals created with this procedure will be automatically degenerated to ordinary numbers if their width is less than `Interval.__MIN_WIDTH`

In [16]:
eps = 1e-5

print('Created via constructor: {}'.format(Interval(1 - eps, 1 + eps)))
print('Created via Interval.create_valid_interval procedure: {}'.format(Interval.create_valid_interval(1 - eps, 1 + eps)))

Created via constructor: [0.99999; 1.00001]
Created via Interval.create_valid_interval procedure: 0.99999


# Interval properties

In [26]:
print('Target interval: %s\n' % i1)

print('Extraction of lower bound: %f' % i1.left)
print('Extraction of upper bound: %f' % i1.right)

Target interval: [-1.0; 1.0]

Extraction of lower bound: -1.000000
Extraction of upper bound: 1.000000


In [27]:
print('Target interval: %s\n' % i2)

print('Extraction of middle point: %f' % i2.middle_point)
print('Extraction of width: %f' % i2.width)
print('Extraction of radius: %f' % i2.radius)

Target interval: [-3.0; 5.0]

Extraction of middle point: 1.000000
Extraction of width: 8.000000
Extraction of radius: 4.000000
